In [3]:
import pandas as pd

In [4]:
df = pd.read_csv('s3://adl-core-sagemaker-studio/external/Hype/UPN_Search_2019_to_current.csv', on_bad_lines='skip')

/opt/conda/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3524: DtypeWarning: Columns (0,4) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [5]:
df.shape

(12810850, 5)

In [6]:
df.head()

,person_internal_id,client_id,platform_id,search_text,search_results
0,NaN,10197,10197,I’d card,44
1,248040081.0,5888,5888,loans outstanding,14
2,NaN,7538,7538,My%20impact,0
3,115130073.0,16581,16581,leave absence,206
4,NaN,567,567,ROLLOVER,26


In [7]:
df2 = df[['search_text']]

In [8]:
df2.head()

,search_text
0,I’d card
1,loans outstanding
2,My%20impact
3,leave absence
4,ROLLOVER


In [9]:
df2 = df2.dropna()

In [10]:
import re

In [11]:
df2['search_text_clean'] = df2['search_text'].map(lambda x: re.sub('[,\.!?%#]', ' ', x))

In [12]:
df2['search_text_clean'] = df2['search_text'].map(lambda x: x.lower())

In [13]:
df2.head()

,search_text,search_text_clean
0,I’d card,i’d card
1,loans outstanding,loans outstanding
2,My%20impact,my%20impact
3,leave absence,leave absence
4,ROLLOVER,rollover


In [14]:
df2 = df2.drop(columns=['search_text'],axis=1)

In [15]:
df2 = df2.rename(columns={'search_text_clean':'search_text'})

In [16]:
df2.head()

,search_text
0,i’d card
1,loans outstanding
2,my%20impact
3,leave absence
4,rollover


In [17]:
import gensim
from gensim.utils import simple_preprocess
import nltk

In [18]:
stop_words = pd.read_csv('nltk_data/english')

In [19]:
#deacc = True removes punctuation
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))

In [20]:
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) 
             if word not in stop_words] for doc in texts]

In [21]:
data = df2.search_text.values.tolist()
data_words = list(sent_to_words(data))

In [ ]:
# remove stop words
data_words = remove_stopwords(data_words)

In [ ]:
print(data_words[:4])

In [ ]:
import gensim.corpora as corpora
# Create Dictionary
id2word = corpora.Dictionary(data_words)
# Create Corpus
texts = data_words
# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

In [ ]:
print(corpus[:1])

In [ ]:
# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100)
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  

bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

# trigram example
print(trigram_mod[bigram_mod[data_words[0]]])

In [ ]:
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [ ]:
# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)

In [ ]:
# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)

In [ ]:
import spacy
nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])

In [ ]:
# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

In [ ]:
from gensim.test.utils import common_corpus, common_dictionary 
import os 
import gensim
#os.environ.update({'MALLET_HOME': '#!new_mallet/mallet-2.0.8/'})
#os.environ.get('MALLET_HOME', 'Not Set')
os.environ["MALLET_HOME"] = 'mallet-2.0.8/'
mallet_path = 'mallet-2.0.8/bin/mallet'
ldamallet = gensim.models.wrappers.LdaMallet(mallet_path=mallet_path,corpus=corpus, num_topics=15, id2word=id2word) #,prefix='temp_file_', workers=4

In [ ]:
print(ldamallet.show_topics(formatted=False))

In [ ]:
# Compute Coherence Score
coherence_model_ldamallet = CoherenceModel(model=ldamallet, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_ldamallet = coherence_model_ldamallet.get_coherence()
print('\nCoherence Score: ', coherence_ldamallet)

In [ ]:
def compute_coherence_values(dictionary, corpus, texts, limit, start=3, step=3):
    coherence_values = []
    model_list = []
    for num_topics in range(start, limit, step):
        model = gensim.models.wrappers.LdaMallet(mallet_path, corpus=corpus, num_topics=num_topics, id2word=id2word)
        model_list.append(model)
        coherencemodel = CoherenceModel(model=model, texts=texts, dictionary=dictionary, coherence='c_v')
        coherence_values.append(coherencemodel.get_coherence())

    return model_list, coherence_values

In [ ]:
model_list, coherence_values = compute_coherence_values(dictionary=id2word, corpus=corpus, texts=[data], start=3, limit=15, step=3)

In [ ]:
import matplotlib.pyplot as plt
limit=15; start=3; step=3;
x = range(start, limit, step)
plt.plot(x, coherence_values)
plt.xlabel("Num Topics")
plt.ylabel("Coherence score")
plt.legend(("coherence_values"), loc='best')
plt.show()

In [ ]:
# Print the coherence scores
for m, cv in zip(x, coherence_values):
    print("Num Topics =", m, " has Coherence Value of", round(cv, 4))